In [10]:
def infoGain(P, N): #Calculate Information gain or Class entropy
    import math
    return -P / (P + N) * math.log2(P / ( P + N)) - N / (P + N) * math.log2(N / (P + N))

In [11]:
def insertNode(tree, addTo, Node):#tree is a dictionary, addTo is the position where to add the Node
    for k, v in tree.items(): #Traversal of tree
        if isinstance(v, dict): #first iteration returns false as tehre are no subtree for root
            tree[k] = insertNode(v, addTo, Node)
    if addTo in tree:    #If d is found then add Node
        if isinstance(tree[addTo], dict):#dict specifies what type of data is passed, here it is dictionary
            tree[addTo][Node] = 'None' #if instance is already present, create another instance
        else:
            tree[addTo] = {Node:'None'} #if no instance is present then create it.
    return tree

In [12]:
def insertConcept(tree, addTo, Node): #Same as insertNode but used to add Concept values
    for k, v in tree.items(): #Traversal of tree
        if isinstance(v, dict):
            tree[k] = insertConcept(v, addTo, Node)
    if addTo in tree:    #If d is found then add Node
        tree[addTo] = Node
    return tree

In [18]:
def getNextNode(data, AttributeList, concept, conceptVals, tree, addTo):
    Total = data.shape[0] #Return number of rows
    if Total == 0: #If Attibutes are empty, then return current value of tree
        return tree
    
    countC = {}
    for cVal in conceptVals:   #If Example is positive then return positive and If negative then return negative
        dataCC = data[data[concept] == cVal]   #Get data for specific concept
        countC[cVal] = dataCC.shape[0]         #Get the count of data for specific concept
#returns count of rows having yes in one iteration and count of rows having no in the second iteration
            
    if countC[conceptVals[0]] == 0: #If all examples are Positive(Not Negative), return single node Positive
        tree = insertConcept(tree, addTo, conceptVals[1])
        return tree
    
    if countC[conceptVals[1]] == 0: #If all examples are Negative (Not Positive), return single node Negative
        tree = insertConcept(tree, addTo, conceptVals[0])
        return tree
#if countC['Yes'] = 0 then insert 'No' in the tree at add to else if countC['No'] = 0 then insert 'Yes' in the tree at add to    
    ClassEntropy = infoGain(countC[conceptVals[1]],countC[conceptVals[0]]) #Calculate Class Entropy for data
    
    Attr = {}         #Attribute dict holding list of possible values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
        
    AttrCount = {}    #Get the attribute values being positive and negative
    EntropyAttr = {}  #Attribute Entropy
    for att in Attr:
        for vals in Attr [att]:
            for c in conceptVals:
                iData = data[data[att] == vals]       #Get data for specific attribute
                dataAtt = iData[iData[concept] == c]  #Get data for specific attribute and concept  
                AttrCount[c] = dataAtt.shape[0]  #Get the count of data for specific attribute and concept
            TotalInfo = AttrCount[conceptVals[1]] + AttrCount[conceptVals[0]] #Total Attribute 
            if AttrCount[conceptVals[1]] == 0 or AttrCount[conceptVals[0]] == 0:
                InfoGain=0
            else:
                InfoGain = infoGain(AttrCount[conceptVals[1]], AttrCount[conceptVals[0]]) #Calculate InfoGain for each  attr
            
            if att not in EntropyAttr:          #Calculate Entropy for each attr
                EntropyAttr[att] = ( TotalInfo / Total ) * InfoGain
            else:
                EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo / Total ) * InfoGain
    
    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g] #Calculate gain

    Node = max(Gain, key = Gain.get)               #Get root node

    tree = insertNode(tree, addTo, Node)
    for nD in Attr[Node]:
        tree = insertNode(tree, Node, nD)    #Insert Attribute value to tree
        newData = data[data[Node] == nD].drop(Node, axis = 1)#Get new data having Attribute of Node(nD) and removing rows with column Node
        AttributeList=list(newData)[:-1]   #New Attribute List
        tree = getNextNode(newData, AttributeList, concept, conceptVals, tree, nD) #Call the function recursively
    return tree

In [1]:
def main():
    from pandas import DataFrame 
    data = DataFrame.from_csv('PlayTennis.csv')    #Read CSV
    print(data)
    AttributeList = list(data)[:-1]            #Get Attribute List
    concept = str(list(data)[-1])              #Get concept list
    conceptVals = list(set(data[concept]))   #Get Concept values
    tree = getNextNode(data, AttributeList, concept, conceptVals, {'root':'None'}, 'root')
    print(tree)   #Call recursive function with initial value of tree and Node as root

In [22]:
main()

       Outlook Temperature Humidity    Wind PlayTennis
slno                                                  
0        Sunny         Hot     High    Weak         No
1        Sunny         Hot     High  Strong         No
2     Overcast         Hot     High    Weak        Yes
3         Rain        Mild     High    Weak        Yes
4         Rain        Cool   Normal    Weak        Yes
5         Rain        Cool   Normal  Strong         No
6     Overcast        Cool   Normal  Strong        Yes
7        Sunny        Mild     High    Weak         No
8        Sunny        Cool   Normal    Weak        Yes
9         Rain        Mild   Normal    Weak        Yes
10       Sunny        Mild   Normal  Strong        Yes
11    Overcast        Mild     High  Strong        Yes
12    Overcast         Hot   Normal    Weak        Yes
13        Rain        Mild     High  Strong         No
{'root': {'Outlook': {'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Rain': {'Wind': {'Strong': 'No', 'Weak': 